In [1]:
import numpy as np

# Checks for the convergence of the two electron integrals

In [6]:
# LOAD PARAMETERS FOR MONTE CARLO INTEGRATION

import MC_integration as mc
import basis_set as bs

L_size = 5
N_walkers = 400
N_steps = 10000
N_skip = 1000
trial_move = 0.6

integrand = bs.two_body_integrand
dimension = 6
sampling = bs.sampling_function

In [8]:
# GENERATE RANDOM STEPS

indices = np.array([1,1,1,1])

init_points = mc.rand_init_point(L_size, dimension, N_walkers)
steps, _, acceptance_ratio = mc.random_walkers(sampling, N_steps, init_points, trial_move)
steps = steps[N_skip:, :, :]

In [34]:
# CHECK IF THERE ARE DISTANCES SMALLER THAN A CERTAIN VALUE (FOR CONVERGENCE OF 1/r12)

r1, r2 = steps[:,:,0:3], steps[:,:,3:6]
dist = np.sum((r1 - r2)**2, axis=2)
dist = dist.flatten() # 1D vector

print("Number of distances smaller than 1E-5: ", len(np.where(dist <= 1E-5)[0]))

Number of distances smaller than 1E-5:  0


In [37]:
# CHECK IF THERE ARE DIVERGENCES IN E_LOCAL()

E_alpha_walkers = mc.MC_average_walkers(integrand, steps, indices)

print(E_alpha_walkers)

[inf inf inf inf inf inf]


In [49]:
# CHECK IF THERE ARE DIVERGENCES IN E_LOCAL()

E_local = integrand(steps, indices)

print("Number of infinities: ", len(np.where(E_local == np.inf)[0]))

Number of infinities:  66


Therefore, the error is in the expression of the integrand (two_body_integrand)

In [38]:
# PREVIOUS INTEGRAND
def two_body_integrand(R, indices):
	"""
	Returns the value of the <pr|g|qs> integrand evaluated at R

	Parameters
	----------

	R: np.ndarray(6,N)
		Positions of the two electrons x1,y1,z1,x2,y2,z2 
		at N diferent coordinates (to evaluate random walkers simultaneously)
	indices: np.ndarray(4)
		Indices of the basis functions 

	Returns
	----------
	I : np.ndarray(N)
		
	"""
	p,q,r,s = indices[0],indices[1],indices[2],indices[3]

	alpha, beta, gamma, delta = basis_radius(p), basis_radius(q), basis_radius(r), basis_radius(s)

	a = alpha + beta
	b = gamma + delta

	R[0:3] = a**(-0.5)*R[0:3]
	R[3:6] = b**(-0.5)*R[3:6]

	r1 = np.sqrt(R[0]**2 + R[1]**2 + R[2]**2)
	r2 = np.sqrt(R[3]**2 + R[4]**2 + R[5]**2)
	r12 = abs(r1-r2)
	I = (a*b)**(-1.5)*HO_wf_3D_basis(R[0:3],p)*HO_wf_3D_basis(R[3:6],r)*(1/r12)*HO_wf_3D_basis(R[0:3],q)*HO_wf_3D_basis(R[3:6],s)/sampling_function(R[0:6])

	return I

In [41]:
# CHECK THE BASIS RADIUS FUNCTION

r = [basis_radius(i+1) for i in range(4)]
print(r)

[2.5066282746309887, 1.5957691216057306, 1.5957691216057306, 1.5957691216057306]


In [74]:
# NEW INTEGRAND
from basis_set import *

def two_body_integrand_new(R, indices):

    p,q,r,s = indices
    alpha, beta, gamma, delta = basis_radius(p), basis_radius(q), basis_radius(r), basis_radius(s)

    a = alpha + beta
    b = gamma + delta

    R[:,:,0:3] = a**(-0.5)*R[:,:,0:3]
    R[:,:,3:6] = b**(-0.5)*R[:,:,3:6]

    r1 = np.sqrt(R[:,:,0]**2 + R[:,:,1]**2 + R[:,:,2]**2)
    r2 = np.sqrt(R[:,:,3]**2 + R[:,:,4]**2 + R[:,:,5]**2)
    r12 = abs(r1-r2)
    
    print("Number of infinities in 1/r12: ", len(np.where(1/r12 == np.inf)[0]))
    print("Number of infinities in 1/sampling_function: ", len(np.where(1/sampling_function_new(R[:,:,0:6]) == np.inf)[0]))
    print("Number of infinities in wf(r1): ", len(np.where(HO_wf_3D_basis_new(R[:,:,0:3],p) == np.inf)[0]))
    print("Number of infinities in wf(r2): ", len(np.where(HO_wf_3D_basis_new(R[:,:,3:6],p) == np.inf)[0]))
    
    I = (a*b)**(-1.5)*HO_wf_3D_basis_new(R[:,:,0:3],p)*HO_wf_3D_basis_new(R[:,:,3:6],r)*(1/r12)*HO_wf_3D_basis_new(R[:,:,0:3],q)*HO_wf_3D_basis_new(R[:,:,3:6],s)/sampling_function_new(R[:,:,0:6])

    return I


def HO_wf_3D_basis_new(R, k, omega_x=OMEGA_X, omega_y=OMEGA_X, omega_z=OMEGA_Z):

    nx, ny, nz = index_to_q_numbers(k)

    psi = HO_wf_3D(R[:,:,0], R[:,:,1], R[:,:,2], nx, ny, nz, omega_x, omega_y, omega_z)

    return psi

def sampling_function_new(R):

    value = 1/(2*np.pi)**3*np.exp(-0.5*np.sum(R**2,axis=-1))

    return value

In [76]:
# CHECK IF THERE ARE DIVERGENCES IN E_LOCAL() WITH NEW INTEGRAND

E_local = two_body_integrand_new(steps, indices)

print("Number of infinities: ", len(np.where(E_local == np.inf)[0]))

Number of infinities in 1/r12:  0
Number of infinities in 1/sampling_function:  0
Number of infinities in wf(r1):  0
Number of infinities in wf(r2):  0
Number of infinities:  0


In [78]:
E_alpha_walkers = np.average(E_local, axis=0)
print(np.average(E_alpha_walkers))

3.523494833996234e-15
